A notebook using AWS Sagemaker ([DeepAR algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html)) to predict COVID-19 positive cases using US time-series data from the public AWS COVID-19 Data Lake

First, we'll be using Athena, so install the appropriate SDK package

In [ ]:
import sys
!{sys.executable} -m pip install PyAthena

We'll need an S3 bucket and a table name to store results and models.

In [ ]:
model_input_table_name = 'a_table_name'
athena_results_bucket = 'your-bucket-name'

Set up a connection to Athena

In [ ]:
from pyathena import connect

athena_results_location = f's3://{athena_results_bucket}/athena/'
cursor = connect(
    s3_staging_dir=athena_results_location,
    region_name='eu-west-2'
).cursor()


Export a query result, structured for the DeepAR model, from Athena. Export format is JSONL by default.

In [ ]:
import uuid
model_input_prefix = f'tables/{model_input_table_name}/{str(uuid.uuid4())}'
model_input_location = f's3://{athena_results_bucket}/{model_input_prefix}/'

cursor.execute(f'DROP TABLE IF EXISTS {model_input_table_name}');

query = f'''
CREATE TABLE {model_input_table_name}
WITH (
  format = 'JSON',
  external_location = '{model_input_location}'
) AS 

WITH input_data AS (
  SELECT
    state,
    date_parse(MIN(date), '%Y%m%d') start_date,
    ARRAY_AGG(positive) positives
  FROM "covid-19".covid_testing_states_daily
  GROUP BY state
)

SELECT
  date_format(start_date, '%Y-%m-%d %H:%i:%s') start,
  positives target
FROM input_data
'''

print(query)

cursor.execute(query);


We'll need to interact with S3

In [ ]:
!{sys.executable} -m pip install boto3;
import boto3
s3 = boto3.resource('s3')

The DeepAR model expects to find data in objects with keys ending `.json.gz` - which our files don't have. Move the objects to appropriate keys.

(I did try and use a different approach - a manifest file, describing the locations - but after a couple of hours I gave up. Instructions not clear enough, debugging basically impossible)

In [ ]:
model_input_s3_keys = s3.Bucket(athena_results_bucket).objects.filter(Prefix=model_input_prefix)
for summary in model_input_s3_keys:
    if summary.key.endswith('.gz') and not summary.key.endswith('.json.gz'):
        print(summary.key)
        s3.Object(athena_results_bucket, summary.key.replace('.gz', '.json.gz')).copy_from(CopySource=f'{athena_results_bucket}/{summary.key}')
        s3.Object(athena_results_bucket, summary.key).delete()

In [ ]:
model_input_s3_keys = s3.Bucket(athena_results_bucket).objects.filter(Prefix=model_input_prefix)
for summary in model_input_s3_keys:
    print(summary.key)

Set up the Sagemaker SDK for the DeepAR model.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

from sagemaker.amazon.amazon_estimator import get_image_uri

image_uri = get_image_uri(boto3.Session().region_name, "forecasting-deepar")

model_output_uri = f's3://{athena_results_bucket}/deepar'

Train a model.

This might take a couple of minutes. Go check out what's happening in the console!

In [ ]:
freq = 'D'
prediction_length = 28
context_length = 56

estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    base_job_name=model_input_table_name.replace('_', '-'),
    output_path=model_output_uri
)

hyperparameters = {
    'time_freq': freq,
    'context_length': str(context_length),
    'prediction_length': str(prediction_length),
    'num_cells': '40',
    'num_layers': '3',
    'likelihood': 'gaussian',
    'epochs': '20',
    'mini_batch_size': '32',
    'learning_rate': '0.001',
    'dropout_rate': '0.05',
    'early_stopping_patience': '10',
}

estimator.set_hyperparameters(**hyperparameters)

train_data = f's3://{athena_results_bucket}/{model_input_prefix}'

data_channels = {
    'train': train_data
}

estimator.fit(inputs=data_channels)

Given the trained model, create an endpoint (an API) to make predictions.

This might take a few minutes. Go check out what's happening in the console!

In [ ]:
job_name = estimator.latest_training_job.name

endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    image_uri=image_uri,
    role=role,
)


Create a Predictor for JSON-formatted inputs using the Sagemaker SDK.

In [ ]:
from sagemaker.predictor import json_serializer

predictor = sagemaker.predictor.RealTimePredictor(
    endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=json_serializer)

Set up an input based on real data from the query results.

In [ ]:
random_state_real_data = [295861.0, 295861.0, 295861.0, 295109.0, 295109.0, 294302.0, 294302.0, 293663.0, 293663.0, 293663.0, 292837.0, 292837.0, 291715.0, 291715.0, 290832.0, 290832.0, 290832.0, 288717.0, 288717.0, 287450.0, 287450.0, 286102.0, 286102.0, 286102.0, 284894.0, 284894.0, 282960.0, 282960.0, 281562.0, 281562.0, 281562.0, 278915.0, 278915.0, 276668.0, 276668.0, 274685.0, 274685.0, 274685.0, 272517.0, 272517.0, 269255.0, 269255.0, 266653.0, 266653.0, 266653.0, 263412.0, 263412.0, 259822.0, 259822.0, 256134.0, 256134.0, 256134.0, 252041.0, 252041.0, 247502.0, 247502.0, 242322.0, 242322.0, 242322.0, 236818.0, 236818.0, 231317.0, 231317.0, 227745.0, 227745.0, 227745.0, 222433.0, 222433.0, 216062.0, 216062.0, 209689.0, 209689.0, 209689.0, 209689.0, 209689.0, 204600.0, 204600.0, 200426.0, 200426.0, 200426.0, 194569.0, 194569.0, 190018.0, 190018.0, 185294.0, 185294.0, 185294.0, 179803.0, 179803.0, 174025.0, 174025.0, 168295.0, 168295.0, 168295.0, 162061.0, 162061.0, 157446.0, 157446.0, 153021.0, 153021.0, 153021.0, 147797.0, 147797.0, 142059.0, 142059.0, 134533.0, 134533.0, 134533.0, 128594.0, 128594.0, 122741.0, 122741.0, 115507.0, 115507.0, 115507.0, 109225.0, 109225.0, 103553.0, 103553.0, 97633.0, 97633.0, 97633.0, 92215.0, 92215.0, 89227.0, 89227.0, 85181.0, 85181.0, 85181.0, 82045.0, 82045.0, 78676.0, 78676.0, 76230.0, 76230.0, 76230.0, 74456.0, 74456.0, 72968.0, 72968.0, 70855.0, 70855.0, 70855.0, 69155.0, 69155.0, 67862.0, 67862.0, 65807.0, 65807.0, 65807.0, 63952.0, 63952.0, 62708.0, 62708.0, 61111.0, 61111.0, 61111.0, 59749.0, 59749.0, 58629.0, 58629.0, 56592.0, 56592.0, 56592.0, 55226.0, 55226.0, 53959.0, 53959.0, 52285.0, 52285.0, 52285.0, 50870.0, 50870.0, 49899.0, 49899.0, 48386.0, 48386.0, 48386.0, 47410.0, 47410.0, 46914.0, 46914.0, 45220.0, 45220.0, 45220.0, 43940.0, 43940.0, 42612.0, 42612.0, 41048.0, 41048.0, 41048.0, 39937.0, 39937.0, 38401.0, 38401.0, 36856.0, 36856.0, 36856.0, 35890.0, 35890.0, 35167.0, 35167.0, 33885.0, 33885.0, 33885.0, 32547.0, 32547.0, 31730.0, 31730.0, 30638.0, 30638.0, 30638.0, 29717.0, 29717.0, 28876.0, 28876.0, 27812.0, 27812.0, 27812.0, 26870.0, 26870.0, 26172.0, 26172.0, 25109.0, 25109.0, 25109.0, 24104.0, 24104.0, 23334.0, 23334.0, 21965.0, 21965.0, 21965.0, 20933.0, 20933.0, 20058.0, 20058.0, 18611.0, 18611.0, 18611.0, 17618.0, 17618.0, 16901.0, 16901.0, 15919.0, 15919.0, 15919.0, 14990.0, 14990.0, 14443.0, 14443.0, 13538.0, 13538.0, 13538.0, 12970.0, 12970.0, 12465.0, 12465.0, 12059.0, 12059.0, 12059.0, 11681.0, 11681.0, 11419.0, 11419.0, 11047.0, 11047.0, 11047.0, 10812.0, 10812.0, 10650.0, 10650.0, 10393.0, 10393.0, 10393.0, 10170.0, 10170.0, 10011.0, 10011.0, 9719.0, 9719.0, 9719.0, 9337.0, 9337.0, 9218.0, 9218.0, 8958.0, 8958.0, 8958.0, 8539.0, 8539.0, 8340.0, 8340.0, 7886.0, 7886.0, 7886.0, 7468.0, 7468.0, 7116.0, 7116.0, 6948.0, 6751.0, 6501.0, 6144.0, 5734.0, 5458.0, 5245.0, 5030.0, 4746.0, 4449.0, 4238.0, 3738.0, 3491.0, 3328.0, 3174.0, 3056.0, 2777.0, 2482.0, 2211.0, 2025.0, 1986.0, 1849.0, 1790.0, 1705.0, 1588.0, 1494.0, 1426.0, 1376.0, 1337.0, 1268.0, 1166.0, 1106.0, 1046.0, 900.0, 845.0, 747.0, 698.0, 620.0, 552.0, 482.0, 428.0, 368.0, 319.0, 261.0, 202.0, 168.0, 126.0, 98.0, 82.0, 64.0, 55.0, 44.0, 34.0, 16.0, 15.0, 11.0, 8.0, 6.0, 6.0, 4.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
predict_sample = random_state_real_data[0:55]
predict_actual = random_state_real_data[55:55+28]

prediction_input = {
    'instances': [
        {'start': '2020-03-06 00:00:00','target': predict_sample}
    ]
}

print(prediction_input)
print(predict_actual)

Get some predictions!

In [ ]:
import json
prediction_response = json.loads(predictor.predict(prediction_input).decode('utf-8'))
print(prediction_response)

Compare the predictions with the actual values.

In [ ]:
predictions = prediction_response['predictions'][0]['mean']

print('diff, prediction, actual')
for prediction, actual in zip(predictions, predict_actual):
    diff = f'{100 * (prediction - actual)/prediction:.1f}%'
    print(diff, int(prediction), int(actual))

These predictions are pretty poor. If there's time, can you improve them?

Delete the endpoint when you're done - they are always-on servers so they cost money!

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name)

We'll also delete the other resources we created.

In [ ]:
sagemaker_session.delete_endpoint_config(endpoint_name)

In [ ]:
sagemaker_session.delete_model(endpoint_name)

In [ ]:
cursor.execute(f'DROP TABLE {model_input_table_name}');